In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Impor the OpenWeatherMap API key
from api_key import key
from geoapi import geoapify_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy



In [36]:
#https://api.census.gov/data/2019/acs/acs5?get=NAME,group(B01003)&for=city:*&key=YOUR_API_KEY

import requests
import json
# Set base URL
base_url = "https://api.census.gov/data"
year = '/2022'
data_name = '/acs/acs1/subject'
#all of the B02015 table ID data for all states
#qry = '?get=group(S1903)&for=place'
qry = "?get=NAME,S1903_C03_001E&for=zip code tabulation area:07003"
key = str('&key='+ key)

    # Make an API request using the params dictionary
test = str(base_url + year + data_name + qry + key)
test
response = requests.get(base_url + year + data_name + qry + key)
result = response.json()
print (json.dumps(result, indent=4))


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [20]:
#https://www.pgpf.org/blog/2023/11/income-and-wealth-in-the-united-states-an-overview-of-recent-data#:~:text=According%20to%20the%20ASEC%2C%20median,the%20pandemic%20and%20subsequent%20recession.
median_income = 74580
# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    
    # Convert JSON data to DataFrame
    df = pd.DataFrame(data[1:], columns=data[0:])
    
    # Save the DataFrame to a CSV file
    df.to_csv("../data/census_data.csv", index=False)
    print("Data saved to census_data.csv")
else:
    print("Error:", response.status_code)

Data saved to census_data.csv


In [29]:
# Convert the cities weather data into a Pandas DataFrame
city_eco_data_df  = pd.read_csv('../data/census_data.csv')

# Splitting the 'NAME' column into two columns 'City' and 'State'
city_eco_data_df[['City', 'State']] = city_eco_data_df['NAME'].str.split(', ', expand=True)
city_eco_data_df.drop(columns=['NAME'], inplace=True)

# Show Record Count
city_eco_data_df.count()
city_eco_data_df.head()

,S1903_C03_001E,state,place,City,State
0,54839,1,3076,Auburn city,Alabama
1,39326,1,7000,Birmingham city,Alabama
2,53929,1,21184,Dothan city,Alabama
3,103194,1,35896,Hoover city,Alabama
4,68930,1,37000,Huntsville city,Alabama


In [37]:
## splitting the eco median data into low income and high income..
city_eco_data_df = pd.DataFrame(city_eco_data_df)

# Splitting the DataFrame into two based on the condition
threshold_value = 74580
df_above_threshold = city_eco_data_df[city_eco_data_df['S1903_C03_001E'] > threshold_value]
df_below_threshold = city_eco_data_df[city_eco_data_df['S1903_C03_001E'] <= threshold_value]

df_above_threshold, df_below_threshold


(      S1903_C03_001E  state  place                    City       State
 3             103194      1  35896             Hoover city     Alabama
 8             100751      2   3000  Anchorage municipality      Alaska
 10             95235      4   7940            Buckeye city     Arizona
 11             75650      4  10670        Casas Adobes CDP     Arizona
 12             98664      4  12000           Chandler city     Arizona
 ...              ...    ...    ...                     ...         ...
 1269          109848     53  65922          South Hill CDP  Washington
 1272           80784     53  70000             Tacoma city  Washington
 1273           76799     53  74060          Vancouver city  Washington
 1275           75262     55   2375           Appleton city   Wisconsin
 1284           74675     55  84250           Waukesha city   Wisconsin
 
 [652 rows x 5 columns],
       S1903_C03_001E  state  place                  City        State
 0              54839      1   3076   

In [38]:
def get_coordinates(city_name, api_key):
    base_url = "https://api.geoapify.com/v1/geocode/search"
    params = {
        "text": city_name,
        "apiKey": api_key
    }

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['features']:
            # Extracting the first result's coordinates
            coordinates = data['features'][0]['geometry']['coordinates']
            return coordinates
        else:
            return "No results found"
    else:
        return "Error in API request"

# Example usage
api_key = geoapify_key # Replace with your actual Geoapify API key
city_name = "New York"
coordinates = get_coordinates(city_name, api_key)
print(f"Coordinates of {city_name}: {coordinates}")

Coordinates of New York: [-74.0060152, 40.7127281]



# Empty list to store the response data
response_data = []

# Iterating over the city names
for city in city_eco_data_df['City']:
    # Construct the API request URL
    url = f"https://api.geoapify.com/v1/geocode/search?text={city}&apiKey={geoapify_key}"

    # Make the API request
    response = requests.get(url)

    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        # Append the data to the list
        response_data.append(data)
        print(f"The data being pulled: {response_data}")
    else:
        print(f"Error fetching data for {city}")

# Creating a DataFrame from the collected data
geoapify_df = pd.DataFrame(response_data)

# Display the DataFrame
print(geoapify_df.head())

###########################################################################################




In [ ]:
# Join the DataFrames on the city names
merged_df = city_eco_data_df.merge(geoapify_df, left_on='NAME', right_on='City')

# Splitting the merged DataFrame into two based on the condition
threshold_value = 74580
df_above_threshold = merged_df[merged_df['S1903_C03_001E'] > threshold_value]
df_below_threshold = merged_df[merged_df['S1903_C03_001E'] <= threshold_value]

df_above_threshold, df_below_threshold

In [ ]:
#PLACES Search with coords from geocode and census data DFs..       
##still in progess

radius = 16093.4  # 10 miles in meters

# Initialize an empty DataFrame
df = pd.DataFrame(columns=["City", "Commercial Weapons Count"])

# Iterate through each city
for city, lat, lon in cities:
    params = {
        "apiKey": geoapify_key,
        "categories": "commercial.weapon",
        "filter": f"circle:{lon},{lat},{radius}",
        "limit": 1  # Adjust as needed
    }
    
    response = requests.get("https://api.geoapify.com/v2/places", params=params)
    
    if response.status_code == 200:
        data = response.json()
        count = len(data['features'])  # Count of commercial weapons
        df = df.append({"City": city, "Commercial Weapons Count": count}, ignore_index=True)
    else:
        print(f"Error fetching data for {city}")

# Save the DataFrame to a CSV file
df.to_csv("city_commercial_weapons.csv", index=False)

print("Data saved to city_commercial_weapons.csv")
